In [1]:
import pandas as pd
import json

from sklearn.model_selection import train_test_split

In [2]:
system_sentiment = 'Do sentiment analysis on user text and return 3 labels: Positive, Negative or Neutral.'
system_tone = 'Do tone analysis on user text and return 5 labels: Kızgın, Korku, Mutlu, Sürpriz or Üzgün.'

In [3]:
def format_training_sample(system_message, user_message, assistant_message):
    new_sample = {
                        "messages": [
                            {"role": "system", "content": system_message}, 
                            {"role": "user", "content": user_message}, 
                            {"role": "assistant", "content": assistant_message}
                        ]
                    }
    return new_sample

In [4]:
df2 = pd.read_csv('../data/TurkishTweets.csv').drop_duplicates()
df2

,Tweet,Etiket
0,Allah'ım çıldıracağım. Yemin ederim çıldıracağ...,kızgın
1,site ne zaman çalıştıda ürün stokları bitti di...,kızgın
2,Sebebi neydi ki diye bağıracağım şimdi az kaldı,kızgın
3,Black friday ne amk bu yıl çıkardınız bu bokud...,kızgın
4,kısa kısa cevaplar vericeksen niye yazıyorsun ...,kızgın
...,...,...
3995,Aynı şeyleri yapmaktan sıkıldım,üzgün
3996,"Olacağı varsa oluyor ya hani, biteceği varsa d...",üzgün
3997,Çok şey istiyorum ama sonu hep fiyasko MUTSUZUM,üzgün
3998,Garip bir şekilde canım çok yanıyor moralim bozuk,üzgün


In [5]:
df2 = df2.dropna()
df2.isnull().sum()

Tweet     0
Etiket    0
dtype: int64

In [6]:
df2['Etiket'].unique()

array(['kızgın', 'korku', 'mutlu', 'surpriz', 'üzgün'], dtype=object)

In [7]:
df2["Etiket"].replace({"kızgın": "Kızgın", "korku": "Korku", "mutlu": "Mutlu", 
                        "surpriz": "Sürpriz", "üzgün": "Üzgün"}, inplace=True)
df2['Etiket'].unique()

C:\Users\assas\AppData\Local\Temp\ipykernel_4968\1684546279.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2["Etiket"].replace({"kızgın": "Kızgın", "korku": "Korku", "mutlu": "Mutlu",
C:\Users\assas\AppData\Local\Temp\ipykernel_4968\1684546279.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Etiket"].replace({"kızgın": "Kızgın",

array(['Kızgın', 'Korku', 'Mutlu', 'Sürpriz', 'Üzgün'], dtype=object)

In [8]:
df2.columns = ['Tweets', 'Label']
df2.head(5)

,Tweets,Label
0,Allah'ım çıldıracağım. Yemin ederim çıldıracağ...,Kızgın
1,site ne zaman çalıştıda ürün stokları bitti di...,Kızgın
2,Sebebi neydi ki diye bağıracağım şimdi az kaldı,Kızgın
3,Black friday ne amk bu yıl çıkardınız bu bokud...,Kızgın
4,kısa kısa cevaplar vericeksen niye yazıyorsun ...,Kızgın


In [9]:
X_train, X_test, y_train, y_test = train_test_split(df2["Tweets"], df2["Label"], test_size=0.1, stratify=df2["Label"])

In [10]:
train_json = []
for index, value in X_train.items():
 train_json.append(format_training_sample(system_tone, value, y_train[index]))
train_json

[{'messages': [{'role': 'system',
    'content': 'Do tone analysis on user text and return 5 labels: Kızgın, Korku, Mutlu, Sürpriz or Üzgün.'},
   {'role': 'user',
    'content': 'Ben bu mavi balina olayını yeni duydum şok oldum aklım almıyor bir insan oyunu bırakmayı nasıl başaramaz.'},
   {'role': 'assistant', 'content': 'Sürpriz'}]},
 {'messages': [{'role': 'system',
    'content': 'Do tone analysis on user text and return 5 labels: Kızgın, Korku, Mutlu, Sürpriz or Üzgün.'},
   {'role': 'user',
    'content': 'Yazılılar her şeyi engelliyor bıktım bıktım stresinden de kendisinden de'},
   {'role': 'assistant', 'content': 'Üzgün'}]},
 {'messages': [{'role': 'system',
    'content': 'Do tone analysis on user text and return 5 labels: Kızgın, Korku, Mutlu, Sürpriz or Üzgün.'},
   {'role': 'user',
    'content': 'Kararlılığınıza hayranım. Öyle ki, dünyayı yerle bir etmenizden korkuyorum.'},
   {'role': 'assistant', 'content': 'Korku'}]},
 {'messages': [{'role': 'system',
    'content': '

In [11]:
test_json = []
for index, value in X_test.items():
 test_json.append(format_training_sample(system_tone, value, y_test[index]))

In [12]:
with open('../jsonl/tweet-tone-train.jsonl', 'w', encoding='utf8') as jsonl_output:
    for entry in train_json:
        json.dump(entry, jsonl_output, ensure_ascii=False)
        jsonl_output.write('\n')

In [13]:
with open('../jsonl/tweet-tone-test.jsonl', 'w', encoding='utf8') as jsonl_output:
    for entry in test_json:
        json.dump(entry, jsonl_output, ensure_ascii=False)
        jsonl_output.write('\n')

In [14]:
pd.concat([X_train, y_train], axis=1).to_csv('../csv/tweet-tone-train.csv', index=False)
pd.concat([X_test, y_test], axis=1).to_csv('../csv/tweet-tone-test.csv', index=False)